In [2]:
# Resnet Models
import torch
import torch.nn as nn
from numba import jit


class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super().__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels,
            intermediate_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

        self.last = nn.Linear(512 * 4,num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        #x = x.reshape(x.shape[0], -1)
        #x = self.fc(x)

        x = x.view(-1,512 * 4)
        x = self.last(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


def ResNet101(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


def ResNet152(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)

In [3]:
model = ResNet50()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (identity_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

In [4]:
from timeit import default_timer as timer
from tqdm import tqdm

In [5]:
from nvidia.dali.pipeline import pipeline_def
import nvidia.dali.types as types
import nvidia.dali.fn as fn
from nvidia.dali.plugin.pytorch import DALIGenericIterator
import os
import numpy as np

In [6]:
import torch
from torch.utils.data import Dataset
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timeit import default_timer as timer

In [7]:
# @pipeline_def(num_threads=4, device_id=0)
# def get_dali_pipeline():
#     data = np.load("/kaggle/input/crazybig-dataset1e7npz/crazybig_dataset(1e7).npz")
#     X = data['X']
#     Y = data['Y']
#     return X,Y

In [8]:
# train_data = DALIGenericIterator(
#     [get_dali_pipeline(batch_size=16)],
#     ['X', 'Y'],
#     reader_name='Reader'
# )

In [9]:
import torch
from torch.utils.data import Dataset
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from timeit import default_timer as timer
from nvidia.dali.pipeline import pipeline_def
import nvidia.dali.types as types
import nvidia.dali.fn as fn
from nvidia.dali.plugin.pytorch import DALIGenericIterator


class ChessValueDataset(DALIGenericIterator):
    def __init__(self):
        data = np.load("/kaggle/input/crazybig-dataset1e7npz/crazybig_dataset(1e7).npz")
        self.X = data['X']
        self.Y = data['Y']


    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return (self.X[idx],self.Y[idx])



def model_training(device,batch_size,epoch):
    chess_dataset = ChessValueDataset()
    train_loader = torch.utils.data.DataLoader(chess_dataset,batch_size=batch_size,num_workers=2,pin_memory=True,shuffle=True)

    #model = Net()

    model = ResNet50(img_channel=5, num_classes=1)


    optimizer = optim.SGD(model.parameters(),lr = 0.0001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.1)
    device = "cuda"
    if device == "cuda":
        model.cuda()
    model.train()
    loss_history = []
    mean_loss_history = []
    print(len(train_loader))
    for epoch in range(epoch):
        all_loss = 0
        num_loss = 0
        #print(len(train_loader))
        tot = timer()
        for batch in tqdm(train_loader,desc='Epoch training {}'.format(epoch),unit='batch'):
            optimizer.zero_grad(set_to_none=True)
            target = batch[-1]
            target = target.unsqueeze(-1)
            target = target.to('cuda:0', non_blocking=True)
            data = batch[0].to('cuda:0', non_blocking=True)
            data = data.float()
            target = target.float()
            
            loss_fn = nn.MSELoss()
            
#             output = model(data)
#             loss = loss_fn(output,target)

            # Enables autocasting for the forward pass (model + loss)
            with torch.cuda.amp.autocast(dtype=torch.float16):
                output = model(data)
                loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            

            all_loss +=loss.item()
            num_loss += 1
            
            
        
        scheduler.step()
        tot_end = timer()

        print(epoch,":",all_loss/num_loss,"run time:",tot_end - tot)
        loss_history.append(all_loss/num_loss)
        mean_loss_history.append(sum(loss_history)/(epoch+1))
        #plot(loss_history, mean_loss_history)
        torch.save(model.state_dict(),'/kaggle/working/value(resnet50-kaggle-1e7).pth')

In [10]:
# from tqdm import tqdm
# import time
# s = timer()
# chess_dataset = ChessValueDataset()
# train_loader = torch.utils.data.DataLoader(chess_dataset,batch_size=256,pin_memory=True,shuffle=True)
# for data in tqdm(train_loader,position=0,desc='load train dataset'):
#     pass
# e = timer()
# print(e-s)

In [11]:
# from tqdm import tqdm
# import time
# s = timer()
# chess_dataset = ChessValueDataset()
# train_loader = torch.utils.data.DataLoader(chess_dataset,batch_size=256,pin_memory=True,shuffle=True)
# for data in tqdm(train_loader,position=0,desc='load train dataset'):
#     pass
# e = timer()
# print(e-s)

In [12]:
# torch.backends.cudnn.benchmark = True
if __name__ == '__main__':
    model_training('cuda',256,10000)

39064


Epoch training 0:  47%|████▋     | 18251/39064 [17:07<19:19, 17.95batch/s]Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f73f7ed23b0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/weakref.py", line 109, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt
Epoch training 0:  47%|████▋     | 18252/39064 [17:08<19:32, 17.75batch/s]

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2474/2203411486.py", line 3, in <module>
    model_training('cuda',256,10000)
  File "/tmp/ipykernel_2474/576914872.py", line 68, in model_training
    output = model(data)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_2474/3919800004.py", line 101, in forward
    x = self.layer3(x)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/container.py", line 204, in forward
    input = module(input)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    retur

TypeError: object of type 'NoneType' has no len()

In [ ]:
# import torch.nn as nn
# from tqdm import tqdm
# from nvidia.dali.pipeline import pipeline_def
# import nvidia.dali.types as types
# import nvidia.dali.fn as fn
# from nvidia.dali.plugin.pytorch import DALIGenericIterator
# import os
# import numpy as np


# @pipeline_def(num_threads=4, device_id=0)
# def get_dali_pipeline():
#     data = np.load("/kaggle/input/crazybig-dataset1e7npz/crazybig_dataset(1e7).npz")
#     X = data['X']
#     Y = data['Y']
#     return X,Y


# print('get_data')
# train_data = DALIGenericIterator([get_dali_pipeline(batch_size=16)],['X', 'Y'],reader_name='Reader')

# print('train')

# loss_fn = nn.MSELoss()
# model = ResNet50(img_channel=5, num_classes=1)

# all_loss = 0
# num_loss = 0
# #for data in tqdm(train_data,desc='Epoch training {}'.format(epoch),unit='batch'):
# for i, data in enumerate(train_data):
#     x,target = data[0]['X'], data[0]['Y']
#     otput = model(x)
#     loss = loss_fc(output, target)
#     loss.backward()
#     all_loss += loss.item()
#     print(all_loss)